<h1>Utility methods built using Bitcoin RPC </h1>

<h2>Building important Methods and Constants</h2>
<p> Values in Bitcoin network is in Little Endian while most tools accepts and returns Big Endian values. This is taken care in below program</p>
<br>

In [20]:
import hashlib
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
import simplejson as json
import binascii
import base58
import ecdsa
from pycoin.ecdsa.numbertheory import modular_sqrt
import pycoin
import pandas as pd
import sys

# Add correct Bitcoin Address
myaddress = '1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa'

rpc_connection = AuthServiceProxy("http://%s:%s@127.0.0.1:8332"%('alice', 'passw0rd'))

SANTOSIS_IN_BTC = 10**8
BLOCK_REWARD_1 = 50 * SANTOSIS_IN_BTC

N_TIME_1 = 1231006505

nBits_1 = 0x1d00ffff
#TARGET_THRESHOLD_1 = 0x00ffff * (256 ** 26) #0x00ffff * (256 ** (0x1d-3)) = 0x00ffff * (256 ** (29-3)) = 0x00ffff * (256 ** 26)
TARGET_THRESHOLD_1 = (0x00ffff << (8 * (0x1d - 3)))

# Target Difficulty set every 2016 blocks
TARGET_DIFFICULTY_SET_EVERY = 2016

# Every Block Height at which Block Reward becomes Half
BLOCK_REWARD_HALVING = 210000


def hash2LittleEndian2LittleEndian(a:str, b:str):
     # Reverse inputs before and after hashing due to big-endian / little-endian nonsense
     a1 = binascii.unhexlify(a)[::-1]
     b1 = binascii.unhexlify(b)[::-1]
     h = hashlib.sha256(hashlib.sha256(a1 + b1).digest()).digest()
     return binascii.hexlify(h[::-1])

def hashBigEndian2LittleEndian(a: str):
     h = hashlib.sha256(hashlib.sha256(bytes.fromhex(a)).digest()).digest()
     return binascii.hexlify(h[::-1])

def double_sha256d(bstr):
    return hashlib.sha256(hashlib.sha256(bstr).digest()).digest()


<h2>Finding Blockchain Related Information</h2>
<br>

<h4>Number of blocks from genesis block</h4>

In [9]:
def getCurrentBlockHeight():
    current_block_height = rpc_connection.getblockcount()
    return current_block_height

<h4>Current Block Reward</h4>
Mining reward to the miner which finds Nounce

In [10]:
def getCurrentBlockReward():
    block_height = getCurrentBlockHeight()
    block_halving_count = int(block_height / BLOCK_REWARD_HALVING)
    #current_block_reward = BLOCK_REWARD_1 / (2 ** block_halving_count) # we can also use bitwise shift operator
    current_block_reward = (BLOCK_REWARD_1 >> block_halving_count)
    return current_block_reward

<h4>Total Bitcoin mined so far</h4>

In [3]:
def getCurrentBitcoinInCirculation():
    block_height = getCurrentBlockHeight()
    block_halving_count = int(block_height / BLOCK_REWARD_HALVING)
    block_reward = BLOCK_REWARD_1 / SANTOSIS_IN_BTC
    bitcoin_in_circulation = 0
    for block_halfing_index in range(block_halving_count):
        bitcoin_in_circulation += (BLOCK_REWARD_HALVING * block_reward)
#        block_reward = block_reward / 2 # we can also use bitwise shift operator
        block_reward = (block_reward >> 1)
    bitcoin_in_circulation += (block_height % BLOCK_REWARD_HALVING) * block_reward
    return bitcoin_in_circulation

<h4>Total Bitcoins to be mined ever</h4>

In [1]:
def getBitcoinCirculationLimit():
    block_reward = BLOCK_REWARD_1 / SANTOSIS_IN_BTC
    bitcoin_in_circulation = 0
    while block_reward != round((block_reward >> 1), 8):
        bitcoin_in_circulation += (BLOCK_REWARD_HALVING * block_reward)
        block_reward = (block_reward >> 1)
    return bitcoin_in_circulation

<h4>Last Bitcoin Mine date</h4>
Date after which no bitcoin will be mined

In [3]:
def getDateToReachLimit():
    block_reward = BLOCK_REWARD_1 / SANTOSIS_IN_BTC
    block_halving_count = 0
    while round(block_reward, 8) != round((block_reward >> 1), 8):
        block_halving_count += 1
        block_reward = (block_reward >> 1)
    sec_to_mine_zero_reward_block = 10 * BLOCK_REWARD_HALVING * block_halving_count * 60
    unix_sec = N_TIME_1 + sec_to_mine_zero_reward_block
    time_of_zero_reward_block = datetime.datetime.fromtimestamp(unix_sec).strftime('%Y-%m-%d %H:%M:%S')
    return time_of_zero_reward_block

<h4>Current Blockchain size</h4>

In [ ]:
def getCurrentBlockchainSizeInGB():
    blockchain_size = sum(os.path.getsize(f) for f in glob.glob(os.path.join(os.getenv('HOME'),'.bitcoin', 'blocks', 'blk0*.dat')))
    return blockchain_size >> 30

<h2>Finding Block Related Information</h2>
<br>
This section contains methods to generate blockhash to validate if block qualifies required target difficulty and to validate Merkel Root is correct. It also provide method for transaction count, transaction rate for a block, network fees collected etc.

<h4>Target Threshold calculation</h4>
Definition: <i>The goal of the miner is to find a candidate block whose block hash is less than or equal to the target threshold</i><br>
The 'bits' field in the block header encodes a 256-bit unsigned integer called the target threshold using a base 256 version of the scientific notation. Let b1 b2 b3 b4 be the four bytes in nBits. The first byte b1 plays the role of the exponent and the remaining three bytes encode the mantissa. The target threshold T is derived as<br>
T = b2b3b4 × 256**(b1−3)<br>
where b1 and b2 b3 b4 are interpreted as unsigned integers.

In [4]:
def getTargetThreshold(hex_bits: bytes):
        shift = '0x%s' % hex_bits[0:2]
        shift_int = int(shift, 16)
        value = '0x%s' % hex_bits[2:]
        value_int = int(value, 16)
#        target = value_int * 2 ** (8 * (shift_int - 3))
        target = (value_int << (8 * (shift_int - 3)))
        hex_target = hex(target)
        return hex_target

<h4>Network Hashrate Calculation</h4>
Output of the SHA-256 function behaves like a random 256-bit string where each bit is equally likely to be 0 or 1 independently of the other bits, the probability that the block hash falls below the target threshold T for a trial nNonce value is<br>
p = (T+1)/2^256
<br>
Average number of trials = 2^256/(T+1)<br>
Let total network Hashrate is required to achieve this is R<br>
It takes 10 min to mine 1 block which means the target threshold is achieved in 10 min. As R is calculated per seconds. R * 10 * 60 is required average number of trials. <br>So R is calculated as<br>
R = 2^256/((T+1) * 600)

In [5]:
def getNetworkHashRate(block_height: int):
        block = getBlock(block_height)
        target_threshold = int(getTargetThreshold(block['bits']), 16)
#        network_hashrate = (2 ** 256) / ((target_threshold + 1) * 600)
        network_hashrate = (1 << 256) / ((target_threshold + 1) * 600)
        return network_hashrate

<h4>Difficulty Calculation</h4>
Definition: <i>Difficulty is calculated as a ratio between intial target threshold and current target threshold</i>

In [6]:
def getDifficulty(block_height: int):
        block = getBlock(block_height)
        target_threshold = int(getTargetThreshold(block['bits']), 16)
        difficulty = TARGET_THRESHOLD_1 / target_threshold

        return difficulty

<h4>Block Header Hash Calculation</h4>

In [16]:
def getBlockHeaderHash(block_header_in_hex):
    header_hash = hashBigEndian2LittleEndian(block_header_in_hex)
    return header_hash

<h4>Next Target Threshold Calculation</h4>
Next Target Threshold is calculated based on average block time in recent 2016 blocks and the current Target Threshold.<br>
T_new = (T_old × Measured duration for finding 2,016 blocks in seconds) / (2016 × 600)

In [17]:
def calculateNextTargetThreashold(block_height: int):
        # no change in target threshold
        if (block_height + 1) % 2016 != 0:
                block = getBlock(block_height)
                target_threshold = int(getTargetThreshold(block['bits']), 16)
                return target_threshold

        n =  (block_height + 1) / 2016
        b1 = 2016 * (n - 1)
        b2 = 2016 * n - 1
        block_t1 = getBlock(b1)
        block_t2 = getBlock(b2)
        t1 = block_t1['time']
        t2 = block_t2['time']
        target_threshold_old = int(getTargetThreshold(block_t2['bits']), 16)
        time_diff = t2 - t1
        target_threshold_new = (target_threshold_old * time_diff) / (2016 * 600)
        return target_threshold_new


<h4>Validate Merkle Tree Root From Transaction Hash List</h4>

In [9]:
def getBlockHash(block_height: int):
        block_hash = rpc_connection.getblockhash(block_height)
        return block_hash
    
def getBlock(block_height: int):
        block_hash = getBlockHash(block_height)
        block = rpc_connection.getblock(block_hash)
        return block

def build_merkle_root(hash_list: list):
        if len(hash_list) < 2:
            return hash_list[0]
        new_hash_list = []

        # Process pairs. For odd length, the last is skipped
        for i in range(0, len(hash_list) - 1, 2):
            new_hash_list.append(hash2LittleEndian2LittleEndian(hash_list[i], hash_list[i + 1]))

        # odd, hash last item twice
        if len(hash_list) % 2 == 1:
            new_hash_list.append(hash2LittleEndian2LittleEndian(hash_list[-1], hash_list[-1]))

        return build_merkle_root(new_hash_list)
    
def validate_merkle_tree_root(block_height: int):
    block = getBlock(block_height)
    merkel_root_in_block_header = block['merkleroot']
    calculated_merkel_tree_root = build_merkle_root(block['tx'])
    if merkel_root_in_block_header == calculated_merkel_tree_root:
        return True

<h4>Get Transaction Count in a Block</h4>
Please refer to Block Structure

In [14]:
def getBlockHash(block_height: int):
        block_hash = rpc_connection.getblockhash(block_height)
        return block_hash
    
def getBlockInHex(block_height: int):
        block_hash = getBlockHash(block_height)
        block = rpc_connection.getblock(block_hash, False)
        return block

def getTxnCountInBlock(block_height: int):
        block_hex = getBlockInHex(block_height)
        indicator = int(block_hex[160:162], 16)

        if indicator < 0xfd:
                txn_count_str = block_hex[162:164]
        elif indicator == 0xfd:
                txn_count_str = block_hex[162:166]
        elif indicator == 0xfe:
                txn_count_str = block_hex[162:170]
        else:
                txn_count_str = block_hex[162:178]

        txn_count = int(bytes.decode(binascii.hexlify(binascii.unhexlify(txn_count_str)[::-1])), 16)
        return txn_count

<h4>Get Actual Block Reward</h4>
Miner gets Current Block reward based on regular halving of block reward after every 210000 blocks and network fees attached to each transactions added to the mined block

In [ ]:
def getActualBlockReward(block_height: int):
        block = getBlock(block_height)
        for txn_hash in getAllTxnsInBlock(block):
                block_reward = 0.0
                txn = getTransactionFromHash(txn_hash)
                block_reward_is_set = False
                block_reward = 0.0
                for vin in txn['vin']:
                        if 'coinbase' in vin:
                                block_reward_is_set = True
                                for vout in txn['vout']:
                                        block_reward += float(vout['value'])
                        if block_reward_is_set == True:
                                break
                if block_reward_is_set == True:
                        break
        return block_reward

<h2>Block and Transaction Parsing</h2>
Raw blocks are stored in files in default location $HOME/.bitcoin/blocks<br>
This section parse one of the file to create JSON structure.<br>


<h2>Script Parser</h2>

<h3>Utility Methods for Script Parsing</h3>

<h4>Get Hash160 encoded Public Key from Bitcoin Address</h4>

In [12]:
def getHash160FromAddress(address: str):
        hash160_of_addr = bytes.decode(binascii.hexlify(base58.b58decode_check(address)))[2:]
        return hash160_of_addr

<h4>Get Address from Public Key</h4>

In [18]:
def double_sha256d(bstr):
    return hashlib.sha256(hashlib.sha256(bstr).digest()).digest()

def convertPKHToAddress(prefix, addr):
    data = prefix + addr
    return base58.b58encode(data + double_sha256d(data)[:4])

def pubkeyToAddress(pubkey_hex):
        pubkey = bytearray.fromhex(pubkey_hex)
        round1 = hashlib.sha256(pubkey).digest()
        h = hashlib.new('ripemd160')
        h.update(round1)
        pubkey_hash = h.digest()
        return convertPKHToAddress(b'\x00', pubkey_hash)

<h4>Get Full Public Key from Compressed Public Key</h4>
Bitcoin uses ECDSA (Elliptic Curve Digital Signature Algorithm) for signing and signature verification.<br>
The elliptic curve domain parameters over Fp associated with a Koblitz curve secp256k1 are specified by the sextuple T = (p,a,b,G,n,h) where the finite field Fp is defined by:<br>
p = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F = 2256 - 232 - 29 - 28 - 27 - 26 - 24 - 1<br><br>
The curve E: y^2 = x^3+ax+b over Fp is defined by:<br>
Here a = 0 and b = 7<br>
So y^2 = x^3 + 7<br><br>
Compressed format has prefix:<br>
0x02+256_bit_x when y is even<br>
0x03+256_bit_x when y is odd<br><br>
Uncompressed format has prefix: <br>
0x04+256_bit_x+256_bit_y

In [ ]:
def getFullPubKeyFromCompressed(x_str: str):
        prefix = x_str[0:2]
        print("prefix = %s" % (prefix))
        x_str = x_str[2:]
        x = int(x_str, 16)
        print("x = \t\t%x" % (x))
        p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
        y_squared = (x**3 + 7) % p
        y = modular_sqrt(y_squared, p)
        y_str = "%x" % y
        print("y_str before = \t%s" % (y_str))
        y_is_even = (int(y_str[-1], 16) % 2 == 0)
        if prefix == "02" and y_is_even == False or prefix == "03" and y_is_even == True:
                y = p - y
                y_str = "%x" % y
        if len(y_str) % 2 == 1:
                y_str = "0" + y_str
        print("y_str after = \t%s" % (y_str))
        return "04" + x_str + y_str

<h4>Verify Signature of Raw Transaction</h4>
ECDSA Key is generated from public key. This Key is used to verify signature using double hashed public key.

In [ ]:
def sigcheck(sig: str, pubkey: str, raw_txn: str):
        hashval = binascii.hexlify(hashlib.sha256(bytes.fromhex(raw_txn)).digest())
        print("hash val = %s" % (hashval))
        txn_sha256 = bytes.decode(hashval)
        print("txn_sha256 = %s" % (txn_sha256))

        prefix = pubkey[0:2]
        if prefix == "02" or prefix == "03":
                pubkey = getFullPubKeyFromCompressed(pubkey)[2:]
        elif prefix == "04":
                pubkey = pubkey[2:]

        print("full public key = %s" % pubkey)
        sig_b = bytes.fromhex(sig)
        txn_sha256_b = bytes.fromhex(txn_sha256)
        vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(pubkey), curve=ecdsa.SECP256k1)
        # We only provide single hashed txn not double because verify method again does hashing of this.
        if vk.verify(sig_b, txn_sha256_b, hashlib.sha256) == True: # True
                print("Signature is Valid")
        else:
                print("Signature is not Valid")

<h3>Script Parser Implementation</h3>

<h4>P2PKH</h4>

<h4>P2SH</h4>

<h4>P2WPKH</h4>

<h4>P2WSH</h4>

<h2>Unspent Transaction Output (UTXO)</h2>

<h2>Create, Sign and Publish Transaction</h2>

<h2>Metrics and Graphs</h2>

<h2>Concept Implementations</h2>

<h3>Escrow Implementation</h3>

<h3>Passphrase generation as replacement for Private Key</h3>